In [ ]:
import sys
import os
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
sys.path.append('/Users/46h/Research/btf/btf-scripts/')
import scan_patterns as sp
from scipy import ndimage

import proplot as pplt
pplt.rc['grid'] = False
CMAP = 'viridis'
pplt.rc['cmap.sequential'] = CMAP
pplt.rc['cmap.discrete'] = False

In [ ]:
datadir = './Diagnostics/Data/Measurements/2022-04-29/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220429125723-x2d-slit.h5'
camname = 'cam34'
ny = 612
nx = 512

In [ ]:
file = h5py.File(join(datadir, filename), 'r')
data = file['scandata']

attrs = data.dtype.names
print('attrs:')
for attr in attrs:
    print('  ', attr)
    
acts = attrs[6:]
print('acts:')
for act in acts:
    print('  ', act)

sdiag = np.array(attrs)[[2, 4, 5]]
print('sdiag:', sdiag)

signame = f'{camname}_integral'
print('signame:', signame)

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
fig, ax = pplt.subplots(figsize=(8, 2))
for act in acts:
    ax.plot(data[:, act], marker=None, label=act)  # plt.plot(tpred,act3,':',color='C2')
ax.legend()
ax.format(xlabel='Step', ylabel='[mm]')

In [ ]:
fig, ax = pplt.subplots()
ax.scatter(data[:, acts[1]], data[:, acts[0]], marker='s', s=50,
           c=np.log10(data[:, signame]), cmap=CMAP, 
           colorbar=True, colorbar_kw=dict(label=f'log10({signame})'))
ax.format(xlabel=acts[1], ylabel=acts[0])
plt.savefig('_output/x2d')

In [ ]:
for item in sdiag:
    print(f'Diagnostic: {item}')
    print('Max = {:.3f}'.format(np.max(data[:, item])))
    print('Min = {:.3f}'.format(np.min(data[:, item])))
    print('Mean = {:.3f}'.format(np.mean(data[:, item])))
    fig, ax = pplt.subplots(figsize=(7.0, 2.0))
    ax.plot(data[:, item], color='black', marker='.', ms=0, label=item)
    ax.format(xlabel='Step', ylabel=item)
    plt.show()

#### Pick frames for background calculation

In [ ]:
bgidx = np.arange(1, 35)

fig, ax = pplt.subplots()
ax.plot(data[:, acts[1]], data[:, acts[0]], alpha=0.2)
ax.scatter(data[:, acts[1]], data[:, acts[0]], marker='s', 
           c=np.log10(data[:, signame]), cmap=CMAP,
           colorbar=True, colorbar_kw=dict(label=f'log10({signame})'))

ax.plot(data[bgidx, acts[1]], data[bgidx, acts[0]], 'rs', label='Background')
ax.legend()
ax.format(xlabel=acts[1], ylabel=acts[0])

## Threshold and background


In [ ]:
bg_lvl_est = np.max(data[bgidx, signame])
print(f'Background level from integral: {bg_lvl_est:.6f}')

thr_lvl_est = bg_lvl_est / np.max(data[:, signame])
print(f'Est. D.R. 10^{np.log10(thr_lvl_est):.3f}')

bg_mean = data[0, f'{camname}_image']
for i in bgidx[1:]:
    bg_mean += data[i, f'{camname}_image']
    
nbg = len(bgidx)
bg_mean = bg_mean / nbg
bg_ima = np.reshape(bg_mean,[nx,ny])

fig, ax = pplt.subplots()
ax.pcolormesh(bg_ima, colorbar=True)
ax.format(title='averaged background images')

In [ ]:
w = data[:, signame]
# thr = 0.043
thr = 0.11
w[w < thr] = 0.0
print(f'signame = {signame}')
print('Est 4D dynamic range 10^%.3f'%np.log10(thr / max(w)))

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(data[:, signame], color='black')
ax.semilogy(w, marker='.', s=4, color='red', lw=0, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame)

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(np.sort(data[:, signame]), color='black')
ax.semilogy(np.sort(w), marker='.', color='red', lw=0, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame,)

### Plot measurement with threshold

In [ ]:
fig, ax = pplt.subplots()
ax.scatter(data[:, acts[1]], data[:, acts[0]], marker='s', s=50,
           c=np.log10(w), cmap=CMAP, 
           colorbar=True, colorbar_kw=dict(label='log10(w)'))
ax.format(xlabel=acts[1], ylabel=acts[0])

### Frame with peak signal

In [ ]:
idx = np.argmax(w)
max_pixel = np.max(data[idx, f'{camname}_image'])

print('peak: ')
for item in list(acts) + list(sdiag):
    print(f'  {item} = {data[idx, item]:.3f} [mm]')
print(f'Max pixel: {max_pixel}')

### Estimate dynamic range by examining frame with peak signal

In [ ]:
camimage = f'{camname}_image'

In [ ]:
idx = np.argmax(w)
im = np.reshape(data[idx, camimage], [nx, ny])
pix_max = np.max(im)
print(f'Max pixel value {pix_max}')

In [ ]:
fig, axes = pplt.subplots(ncols=2, sharey=False)
axes[0].pcolormesh(np.log10(im), colorbar=True)
axes[0].annotate('(logscale)', xy=(1, 1), color='white')
axes[1].plot(np.sum(im, axis=0))
plt.show()

In [ ]:
mask = np.ones(np.shape(im))
mask[400:, :] = 0.0
mask[:100, :] = 0.0
mask[:, 480:] = 0.0
mask[:, :200] = 0.0
immask = im * mask

print('5D DR: 10^%.3f'%(np.log10(np.max(mask * bg_ima) / np.max(immask))))
print('4D DR: 10^%.3f'%(np.log10(np.sum(mask * bg_ima) / np.sum(immask))))

In [ ]:
fig, axes = pplt.subplots(ncols=2, sharey=False)
axes[0].pcolormesh(np.log10(immask))
axes[0].annotate('(logscale)', xy=(1, 1), color='white')
axes[1].plot(np.sum(im, axis=0))
plt.show()

In [ ]:
thr2 = 1e5

plt.figure()
imathr = im.copy()
imathr[imathr < thr2] = 0.0
plt.pcolor(np.log10(imathr))

print('naive 5D DR: 10^%.3f'%(np.log10(thr2/imathr.max())))
print('naive 4D DR: 10^%.3f'%(np.log10(thr2/imathr.sum())))
print('5D DR: 10^%.3f'%(np.log10(bg_ima.max()/im.max())))
print('4D DR: 10^%.3f'%(np.log10(bg_ima.sum()/im.sum())))

#### Threshold after smoothing

In [ ]:
ima = im
ima_smooth = ndimage.gaussian_filter(ima, 5, mode='nearest')

thr2 = 50000.0
plt.figure()
imathr = ima_smooth.copy()
imathr[imathr < thr2] = 0.0
plt.pcolor(np.log10(imathr))

print('naive 5D DR: 10^%.3f'%(np.log10(thr2/imathr.max())))
print('naive 4D DR: 10^%.3f'%(np.log10(thr2/imathr.sum())))
print('5D DR: 10^%.3f'%(np.log10(bg_ima.max()/ima.max())))
print('4D DR: 10^%.3f'%(np.log10(bg_ima.sum()/ima.sum())))

## Measure against new scan boundaries

Make a list of coordinates that have signal.

In [ ]:
pts = np.vstack([data[:, acts[0]], data[:, acts[1]]]).T
idx, = np.where(w > 0.0)
sigpts = pts[idx, :]

In [ ]:
ndim = 2
M = np.identity(ndim)
M[0, 1] = 1.9

# xp
d1_center = 15.0
d1_distance = 24.0
d1_steps = 32

# x
d2_center = 21.0
d2_distance = 17.5
d2_steps = 32

reprate = 5
navg = 10

In [ ]:
# def gen(): 
    
#     center = np.array([slit_dict[keys[i]]['center'] for i in range(ndim)])
#     distance = np.array([slit_dict[keys[i]]['distance'] for i in range(ndim)])
#     nsteps = np.array([slit_dict[keys[i]]['steps'] for i in range(ndim)])
#     stepsize = distance / nsteps

#     start = center -0.5*distance
#     taking_step = [False,]*ndim
#     step_counter = [0,]*ndim
    
#     # -- if sweeper scan, adjust step parameters for first actuator
#     if navg == 0: 
#         sweep_speed = stepsize[0]*reprate
#         stepsize[0] = distance[0]
#         nsteps[0] = 1

#     # -- scan is complete after <npoints> steps
#     npoints = 1
#     for i in range(ndim):
#         npoints *= (nsteps[i]  +1 )

#     # start loop
#     taking_step[0] = True
#     counter = 0
#     lastpoint = start
#     while counter < npoints:
#         # print('%i / %i, %.1f%%'%(counter,npoints,counter*100./npoints))
#         # -- send actuators to starting point
#         # -- this is only needed for stepper scan
#         if (counter == 0) & (navg > 0):
#             counter += 1
#             # -- cast int sheared frame first:
#             nextactpoint = np.matmul(M,lastpoint - center) + center
#             # don't let slit destination exceed min/max specified in slit_dict
#             # skip points that exceed this limit, but otherwise do not change scan
#             skipFlag = False
#             for i in range(ndim):
#                 if (nextactpoint[i] > slit_dict[keys[i]]['max']) or (nextactpoint[i] < slit_dict[keys[i]]['min']):
#                     skipFlag = True
#             if not(skipFlag):    
#                 cmd = list(nextactpoint)
#                 yield cmd, navg
                
#         counter += 1
#         thisstep = [0,]*ndim
#         for i in range(ndim):
#             # -- if end of line has been reached
#             if step_counter[i] >= nsteps[i]:
#                 # -- stop stepping this actuator
#                 taking_step[i] = False
#                 # -- and make sure the next actuator knows it's time to step
#                 taking_step[int(np.mod(i+1,ndim))] = True
#                 # -- and roll over the stepcounter
#                 step_counter[i] = np.mod(step_counter[i],nsteps[i])
#                 # -- while also reversing direction of step
#                 stepsize[i] *= -1
#             # -- if end of line has not been reached
#             elif taking_step[i]:
#                 # -- take the next assigned step
#                 thisstep[i] += stepsize[i]
#                 # -- add one to step_counter
#                 step_counter[i] += 1

#                 # -- now that a step has been taken, stop until your turn comes up again 
#                 # unless you're the first actuator; that guy always steps
#                 if i > 0:
#                     taking_step[i] = False
#                     taking_step[0] = True

#             # -- exit loop if an actuator has stepped
#             if thisstep[i] != 0:
#                 break

#         # -- get startpoint in actuator coordinates
#         startactpoint = np.matmul(M,lastpoint - center) + center
#         # -- progress 1 step
#         lastpoint = lastpoint + thisstep
#         # -- cast into actuator coordinates (sheared frame)
#         nextactpoint = np.matmul(M,lastpoint - center) + center
        
#         ##### deal with points outside slit min/max limits.
#         # -- distinguish between steppers, sweepers
#         isweep = []
#         if navg != 0: # all steppers if averaging is on
#             istep = range(ndim)
#         else: 
#             istep = []; 
#             for j in range(0,ndim):
#                 dxj =  np.abs(nextactpoint[j] - startactpoint[j])
#                 if dxj == 0: # if distance == 0, this point is a stepping point
#                     istep.append(j)
#                 else:
#                     isweep.append(j)
                    
#         # step scan: skip points that exceed this limit, but otherwise do not change scan
#         skipFlag = False
#         for i in istep:
#             if (nextactpoint[i] > slit_dict[keys[i]]['max']) or (nextactpoint[i] < slit_dict[keys[i]]['min']):
#                 skipFlag = True   
#         # sweep scan; this will result in odd behavior if there is > 1 sweeper.
#         for i in isweep:
#             if (startactpoint[i] > slit_dict[keys[i]]['max']):
#                 startactpoint[i] = slit_dict[keys[i]]['max']
#                 print('%i sweep shortened on max end'%counter)
#             if (startactpoint[i] < slit_dict[keys[i]]['min']):
#                 startactpoint[i] = slit_dict[keys[i]]['min']
#                 print('%i sweep shortened on minend'%counter)
#             if (nextactpoint[i] > slit_dict[keys[i]]['max']):
#                 nextactpoint[i] = slit_dict[keys[i]]['max']
#                 print('%i sweep shortened on max end'%counter)
#             if (nextactpoint[i] < slit_dict[keys[i]]['min']):
#                 nextactpoint[i] = slit_dict[keys[i]]['min']
#                 print('%i sweep shortened on minend'%counter)

#         ##### STEPPER SCAN COMMAND
#         if (navg > 0) & (not(skipFlag)):
#             cmd = list(nextactpoint)
#             yield cmd ,navg

#         ##### SWEEPER SCAN COMMAND
#         elif (navg == 0) & (not(skipFlag)): 
#             # -- skip step if sweeper is not moving
#             if thisstep[0] == 0:
#                 continue
#             # -- send command if sweeper is moving on this step
#             else:                
#                 # -- sweeper command
#                 cmd = []
#                 startpt = startactpoint[0]#np.min([startactpoint[0],49])
#                 endpt = nextactpoint[0]#np.min([nextactpoint[0],49])
#                 cmd.append((startpt,endpt,sweep_speed))
#                 # -- commands for the rest of the actuators
#                 for j in range(1,ndim):
#                     # if slope, stepper may have to sweep as well
#                     stepper_speed = sweep_speed * np.abs(nextactpoint[j] - startactpoint[j]) / d1_distance
#                     cmd.append((startactpoint[j],nextactpoint[j],stepper_speed))
#             lastcmd = cmd.copy() # -- copy command in case it needs repeated
#             yield cmd

In [ ]:
def gen(start=0): 
    center = np.array([d1_center, d2_center])
    distance = np.array([d1_distance, d2_distance])
    nsteps = np.array([d1_steps, d2_steps])
    stepsize = distance / nsteps
    counter = 0
        
    ##### STEPPER SCAN COMMANDS
    if navg > 0:             
        for point in sp.grid(center,distance,nsteps): 
            realpoint = np.matmul(M,point-center) + center

            counter += 1
            if counter > start:
                yield realpoint, navg
        
    ##### SWEEPER SCAN COMMANDS
    elif navg == 0:
        sweep_speed = distance[0]/nsteps[0]*reprate
        nsteps[0] = 1

        lastpoint = next(sp.grid(center,distance,nsteps))
        for point in sp.grid(center,distance,nsteps): 
            # -- skip step if sweeper is not moving
            if (point[0] - lastpoint[0]) == 0:
                lastpoint = point.copy()
                continue

            # -- send command if sweeper is moving on this step
            else: 
                # -- first transform to real coordinates
                realpoint = np.matmul(M,point-center) + center
                reallastpoint = np.matmul(M,lastpoint-center) + center

                # -- build up cmd list
                cmd = []
                # -- sweeper command
                cmd.append((reallastpoint[0],realpoint[0],sweep_speed))
                # -- commands for the rest of the actuators
                for j in range(1,ndim):
                    # if slope, stepper may have to sweep as well
                    stepper_speed = sweep_speed * np.abs(realpoint[j] - reallastpoint[j]) / distance[0]
                    cmd.append((reallastpoint[j],realpoint[j],stepper_speed))
            
                lastpoint = point.copy()
            counter+=1
            if counter>start:        
                yield cmd

In [ ]:
# -- run points generator

# -- re-arrange array
#  sweeping
# genpts = np.array(list(gen()))
# newpts = np.zeros([np.shape(genpts)[0]*2,ndim])
# for i in range(ndim):
#     newpts[:,i] = genpts[:,i,0:2].flatten()

# averaging
lgen= list(gen())
newpts = np.zeros([len(lgen), ndim])
for i in range(len(lgen)):
    newpts[i, :] = lgen[i][0]
    
# Un-shear new (generated) points.
center = center = np.array([d1_center, d2_center])
Minv = np.linalg.inv(M)
newpts_n = np.zeros(np.shape(newpts))
for row in range(np.shape(newpts)[0]):
    newpts_n[row, :] = np.matmul(Minv, newpts[row, :] - center) + center
    
# Un-shear signal points.
sigpts_n = np.zeros(sigpts.shape)
for row in range(np.shape(sigpts)[0]):
    sigpts_n[row, :] = np.matmul(Minv, sigpts[row, :] - center) + center

In [ ]:
coords = np.vstack([data[:, acts[0]], data[:, acts[1]]]).T
coords_n = np.zeros(coords.shape)
for row in range(coords.shape[0]):
    coords_n[row, :] = np.matmul(Minv, coords[row, :] - center) + center

In [ ]:
fig, axes = pplt.subplots(ncols=2, sharex=False, sharey=False,
                          wspace=8.0, figwidth=8.0)
for ax, signal, new in zip(axes, [sigpts, sigpts_n], [newpts, newpts_n]):
    ax.scatter(signal[:, 0], signal[:, 1], marker='s', color='black', label='signal')
    ax.plot(new[:, 0], new[:, 1], marker='.', s=4, color='grey', label='scan');
axes.format(xlabel=acts[0], ylabel=acts[1], toplabels=['Sheared', 'Un-sheared'])
axes[0].axvline(49.0, color='red', label='limit')
axes[0].legend(ncols=1)
plt.show()

In [ ]:
# fig, axes = pplt.subplots(ncols=2, sharex=False, sharey=False,
#                           wspace=8.0, figwidth=7.0)
# axes[0].scatter(sigpts[:, 0], sigpts[:, 1], marker='s', color='black', label='signal')
# axes[0].plot(newpts[:, 1], newpts[:, 2], marker='.', s=4, color='grey', label='scan');
# # axes.format(xlabel=acts[0], ylabel=acts[1], toplabels=['Sheared', 'Un-sheared'])
# # xlim = axes[0].get_xlim()
# # ylim = axes[0].get_ylim()
# # for x in [-50.0, 50.0]:
# #     axes[0].axvline(x, color='red')
# #     if x == 50.0:
# #         axes[0].axhline(x, color='red', label='limit')
# #     else:
# #         axes[0].axhline(x, color='red')
# # axes[0].format(xlim=(min(-5, axes[0].get_xlim()[0]), 50.0))
# # axes[0].format(ylim=(min(-5, axes[0].get_ylim()[0]), 50.0))
# axes[0].legend(ncols=1)
# plt.show()

In [ ]:
np.min(newpts, axis=0)

In [ ]:
np.max(newpts, axis=0)